### Using GPU

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6488755551176463180
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3155650150
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14968211924725505902
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [4]:
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

### Getting the current working directory

In [5]:
import os
PATH = os.getcwd()

### Defining the data path

In [6]:
DATA_PATH = os.path.join(PATH, 'data')

print(DATA_PATH)

C:\Work\INSOFE\Study Material\CUTe\CUTe4\Image classification\data


### Getting the list of folders inside data path

In [7]:
data_dir_list = os.listdir(DATA_PATH)

#if '.DS_Store' in data_dir_list :
#    data_dir_list.remove('.DS_Store')
    
print(data_dir_list)

['airplanes', 'BACKGROUND_Google', 'bonsai', 'car_side', 'Faces', 'Faces_easy', 'grand_piano', 'Leopards', 'Motorbikes', 'watch']


### Reading the images and storing them in the list

In [8]:
# !pip install opencv-python

import cv2

In [9]:
img_rows=300
img_cols=300

img_data_list=[]
classes_names_list=[]

for dataset in data_dir_list:
    
    classes_names_list.append(dataset) 
    
    print ('Loading images from {} folder\n'.format(dataset)) 
    
    img_list = os.listdir(DATA_PATH+'/'+ dataset)
    
    for img in img_list:
        input_img = cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))            
        img_data_list.append(input_img_resize)

Loading images from airplanes folder

Loading images from BACKGROUND_Google folder

Loading images from bonsai folder

Loading images from car_side folder

Loading images from Faces folder

Loading images from Faces_easy folder

Loading images from grand_piano folder

Loading images from Leopards folder

Loading images from Motorbikes folder

Loading images from watch folder



### Getting the number of classes

In [10]:
num_classes = len(classes_names_list)
print(num_classes)

10


### Image preprocessiong

In [11]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float16')
img_data /= 255              ############### 0-255, i.e., 2**8 = 256 possibilities is the rance for each individual colour. 
                             ############### By dividing by 255, we get values in the 0.0-1.0 range (normalization) 
                             ############### NN or deep learning required scaled or normalized data


In [12]:
print (img_data.shape)

(3725, 300, 300, 3)


In [13]:
num_of_samples = img_data.shape[0]
print(num_of_samples)

3725


In [14]:
input_shape = img_data[0].shape
print(input_shape)

(300, 300, 3)


In [15]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:800]=0
classes[800:1268]=1
classes[1268:1396]=2
classes[1396:1519]=3
classes[1519:1954]=4
classes[1954:2389]=5
classes[2389:2488]=6
classes[2488:2688]=7
classes[2688:3486]=8
classes[3468:]=9

print(classes)

[0 0 0 ... 9 9 9]


### Converting class labels to numberic using on-hot encoding

In [16]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

print(classes)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


### Shuffling the dataset

In [17]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

### Splitting the dataset into train and test

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [19]:
X_train.shape

(2980, 300, 300, 3)

In [20]:
y_train.shape

(2980, 10)

In [21]:
X_test.shape

(745, 300, 300, 3)

In [22]:
y_test.shape

(745, 10)

In [23]:
del X,Y

### Defining the model

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [25]:
model = Sequential()

model.add(Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (2, 2), activation='relu'))
model.add(Conv2D(256, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(512, (2, 2), activation='relu'))
# model.add(Conv2D(512, (2, 2), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compiling the model

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

### Model Summary

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 32)      416       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 298, 32)      4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 148, 148, 64)      8256      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 64)      16448     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
__________

In [28]:
model.get_config()

{'name': 'sequential_1',
 'layers': [{'class_name': 'Conv2D',
   'config': {'name': 'conv2d_1',
    'trainable': True,
    'batch_input_shape': (None, 300, 300, 3),
    'dtype': 'float32',
    'filters': 32,
    'kernel_size': (2, 2),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d_2',
    'trainable': True,
    'filters': 32,
    'kernel_size': (2, 2),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
 

In [29]:
model.layers[0].get_config()

{'name': 'conv2d_1',
 'trainable': True,
 'batch_input_shape': (None, 300, 300, 3),
 'dtype': 'float32',
 'filters': 32,
 'kernel_size': (2, 2),
 'strides': (1, 1),
 'padding': 'valid',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [30]:
model.layers

In [31]:
model.weights

[<tf.Variable 'conv2d_1/kernel:0' shape=(2, 2, 3, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/kernel:0' shape=(2, 2, 32, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_3/kernel:0' shape=(2, 2, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_3/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_4/kernel:0' shape=(2, 2, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_4/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_5/kernel:0' shape=(2, 2, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2d_5/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2d_6/kernel:0' shape=(2, 2, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2d_6/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2d_7/kernel:0' shape=(2, 2, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv2d_7/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv2

In [32]:
model.layers[0].get_weights()

[array([[[[-0.05312739,  0.00771603, -0.15198979, -0.08276855,
            0.00790112,  0.08279593,  0.15254666,  0.00642884,
           -0.09208238, -0.13611874,  0.16623072,  0.16245534,
            0.03783062, -0.13264404,  0.0011941 , -0.19690314,
           -0.1589872 ,  0.15278484,  0.16625448,  0.10810618,
            0.00695825, -0.02051637, -0.11179407, -0.04035184,
            0.16123037, -0.08878363,  0.05850099,  0.07472514,
           -0.14844204,  0.10631545, -0.02038819, -0.16382623],
          [-0.02990744, -0.16037363,  0.04430591,  0.18876208,
           -0.10094261,  0.16359873,  0.06867896,  0.02016105,
            0.14340247,  0.17276455, -0.01302284, -0.17692295,
            0.14691044, -0.15212646,  0.16768973, -0.15082994,
            0.12239583, -0.15604569, -0.17118935, -0.05961064,
            0.11116363, -0.09179472, -0.01841463, -0.1995491 ,
           -0.11531542,  0.01676546, -0.19879116, -0.02927659,
           -0.01346621,  0.13965403,  0.03518048, -0.0

In [33]:
# For layer 0, checking the filter (get_weights()[0]) and bias (get_weights()[0]))

print(type(model.layers[0].get_weights()))

print(len(model.layers[0].get_weights()))

print(type(model.layers[0].get_weights()[0]))

print(model.layers[0].get_weights()[0].shape)

print(type(model.layers[0].get_weights()[1]))

print(model.layers[0].get_weights()[1].shape)

<class 'list'>
2
<class 'numpy.ndarray'>
(2, 2, 3, 32)
<class 'numpy.ndarray'>
(32,)


In [34]:
#Checking input shape for layer 0

model.layers[0].input_shape

(None, 300, 300, 3)

In [35]:
#Checking output shape for layer 0

model.layers[0].output_shape

(None, 299, 299, 32)

In [36]:
model.layers[0].trainable

True

### Approach1: Data augmentation

In [37]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [38]:
datagen.fit(X_train)

### Approach1: Training/fitting the model on augmented data

In [39]:
# num_epoch = 5
# batch_size = 32

# # hist = model.fit(X_train, y_train, batch_size = batch_size, 
# #                  epochs = num_epoch, verbose = 1, 
# #                  validation_data = (X_test, y_test))

# hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size),\
#                           steps_per_epoch = len(X_train)/batch_size, \
#                           epochs = num_epoch,\
#                           verbose = 1, validation_data = (X_test, y_test))

**Saving and loading the model and weights**

It is not recommended to use pickle or cPickle to save a Keras model.

You can use model.save(filepath) to save a Keras model into a single HDF5 file which will contain:

    the architecture of the model, allowing to re-create the model
    the weights of the model
    the training configuration (loss, optimizer)
    the state of the optimizer, allowing to resume training exactly where you left off.

You can then use keras.models.load_model(filepath) to reinstantiate your model. load_model will also take care of compiling the model using the saved training configuration (unless the model was never compiled in the first place).

In [40]:
# model.save('model.hdf5')
# loaded_model = keras.models.load_model('model.hdf5')

**Using saved model for retraining**

In [41]:
# num_epoch = 10
# batch_size = 32

# hist = loaded_model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size),\
#                           steps_per_epoch = len(X_train)/batch_size, \
#                           epochs = num_epoch,\
#                           verbose = 1, validation_data = (X_test, y_test))

**Saving and loading the retrained model**

In [42]:
#loaded_model.save('loaded_model_augmentation_copy.hdf5')
loaded_model_augmentation_copy = keras.models.load_model('loaded_model_augmentation_copy.hdf5')

Instructions for updating:
Use tf.cast instead.


### Approach1: Evaluating the model

In [44]:
#loaded_model_augmentation_copy
num_epoch = 10
batch_size = 32

score_loaded_test = loaded_model_augmentation_copy.evaluate(X_test, y_test, batch_size=batch_size)

print('Test Loss:', score_loaded_test[0])
print('Test Accuracy:', score_loaded_test[1])

745/745 [==============================] - 8s 10ms/step
Test Loss: 0.8532429583120666
Test Accuracy: 0.7691275168585298


In [45]:
# score_loaded = loaded_model.evaluate(X_test, y_test, batch_size=batch_size)

# print('Test Loss:', score_loaded[0])
# print('Test Accuracy:', score_loaded[1])

In [46]:
test_image_aug = X_test[0:1]
print (test_image_aug.shape)

(1, 300, 300, 3)


In [47]:
print(loaded_model_augmentation_copy.predict(test_image_aug))
print(loaded_model_augmentation_copy.predict_classes(test_image_aug))
print(y_test[0:1])

[[4.0252716e-03 2.8371654e-02 2.2205018e-05 1.1738068e-02 9.8851975e-05
  3.6261059e-07 5.8562942e-05 8.2516703e-14 1.0114807e-01 8.5453701e-01]]
[9]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Approach1: Predict and compute the confusion matrix

In [48]:
# from sklearn.metrics import confusion_matrix

# Y_pred_loaded = loaded_model.predict(X_test)
# print(Y_pred_loaded)

In [49]:
from sklearn.metrics import confusion_matrix

Y_pred_loaded_aug = loaded_model_augmentation_copy.predict(X_test)
print(Y_pred_loaded_aug)

[[4.02526557e-03 2.83716507e-02 2.22049766e-05 ... 8.25167029e-14
  1.01148047e-01 8.54537010e-01]
 [2.36970577e-02 1.49158180e-01 1.37636922e-02 ... 8.12652706e-06
  7.20552683e-01 7.62639940e-02]
 [1.12401836e-01 8.38459060e-02 7.10340776e-03 ... 6.89990120e-04
  3.54710296e-02 4.20511141e-02]
 ...
 [1.21915973e-05 1.79868529e-03 7.38199617e-07 ... 4.38681724e-07
  4.01580201e-06 5.86802234e-05]
 [6.74744719e-04 1.23054180e-02 2.73327805e-05 ... 1.74491799e-06
  2.52595637e-04 2.09580190e-04]
 [7.37014576e-04 3.69170215e-03 9.28470308e-06 ... 1.12449316e-05
  1.03542998e-05 1.14801849e-04]]


In [50]:
# y_pred_loaded = np.argmax(Y_pred_loaded, axis=1)
# print(y_pred_loaded)

# # Class description:
# # 0 = airplanes
# # 1 = BACKGROUND_Google
# # 2 = bonsai
# # 3 = car_side
# # 4 = Faces
# # 5 = Faces_easy
# # 6 = grand_piano
# # 7 = Leopards
# # 8 = Motorbikes
# # 9 = watch

In [51]:
y_pred_loaded_aug = np.argmax(Y_pred_loaded_aug, axis=1)
print(y_pred_loaded_aug)

# Class description:
# 0 = airplanes
# 1 = BACKGROUND_Google
# 2 = bonsai
# 3 = car_side
# 4 = Faces
# 5 = Faces_easy
# 6 = grand_piano
# 7 = Leopards
# 8 = Motorbikes
# 9 = watch

[9 8 3 0 9 1 0 5 8 8 6 0 0 0 4 7 8 3 1 5 4 4 9 4 5 4 1 0 1 5 5 4 5 1 1 7 5
 4 1 3 1 8 8 7 6 5 0 4 4 1 0 5 0 3 2 8 9 0 0 8 1 5 0 1 8 4 5 1 1 1 9 8 3 1
 1 4 8 3 1 1 0 3 4 0 8 1 3 0 8 0 1 5 1 4 0 5 1 1 8 1 0 0 1 0 8 0 5 1 8 5 5
 0 5 1 8 5 8 5 3 3 9 8 7 3 1 1 5 5 4 4 0 4 3 1 3 8 8 1 3 1 0 4 0 1 0 0 0 4
 8 0 4 0 5 1 5 5 5 0 5 4 5 1 8 4 5 8 5 3 0 1 0 3 3 1 1 5 5 3 3 1 1 5 0 0 0
 0 8 8 0 6 0 3 8 0 0 3 0 5 8 4 3 1 4 8 3 8 8 3 8 4 8 4 1 3 4 0 8 7 5 8 4 1
 0 0 1 9 8 4 5 5 0 4 1 3 3 4 4 1 8 8 4 1 3 0 9 1 8 5 4 4 3 0 9 3 0 2 0 0 8
 8 3 2 3 0 3 0 9 0 8 4 0 1 3 1 8 4 5 0 0 8 2 5 3 1 8 5 5 0 0 8 8 5 0 8 8 1
 4 8 0 0 0 3 1 8 0 0 1 0 8 0 8 0 0 0 1 9 7 3 0 1 8 2 3 0 1 1 0 0 0 3 0 0 0
 0 4 9 1 8 3 3 5 0 4 4 0 4 4 6 3 9 5 3 8 0 5 4 3 0 0 0 8 3 0 8 3 1 5 3 0 1
 1 8 5 0 0 8 8 1 8 1 5 8 1 0 8 0 7 0 1 0 0 8 8 8 1 3 0 9 1 9 1 0 8 0 8 3 4
 6 5 0 0 0 0 0 0 8 8 1 0 5 8 1 1 1 9 0 9 8 8 2 5 1 8 0 0 7 1 5 1 1 5 5 0 8
 5 0 1 0 9 0 8 3 3 1 1 1 4 8 0 5 0 0 0 0 2 5 4 8 0 1 0 6 1 6 8 1 1 4 0 0 5
 5 0 8 1 4 5 3 3 1 0 0 1 

In [52]:
# print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_loaded))

In [53]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_loaded_aug))

[[140  10   0  19   0   0   0   0   2   0]
 [  4  73   0   6   1   0   0   1   1   4]
 [  0   3  15   0   0   0   1   1   0   1]
 [  0   0   0  23   0   0   0   0   0   0]
 [  0  13   0   3  69   7   0   0   0   0]
 [  0   9   0   1   0  86   0   0   0   0]
 [  0   5   0   0   0   0   7   0   0   0]
 [ 27   0   0   2   0   0   0  11   0   0]
 [  2  16   0  15   0   0   0   0 119   0]
 [  1   4   0   4   0   0   4   0   5  30]]


## Approach1: Prediction on Unseen Image

In [54]:


UNSEEN_DATA_PATH = os.path.join(PATH, 'useen_image')

#print(UNSEEN_DATA_PATH)

unseen_data_dir_list = os.listdir(UNSEEN_DATA_PATH)

if '.DS_Store' in data_dir_list :
    data_dir_list.remove('.DS_Store')
    
print(unseen_data_dir_list)

unseen_img_data_list=[]
unseen_img_classes_names_list=[]

for dataset in unseen_data_dir_list:
    
    unseen_img_classes_names_list.append(dataset) 
    
    print ('Loading images from {} folder\n'.format(dataset)) 
    
    unseen_img_list = os.listdir(UNSEEN_DATA_PATH+'/'+ dataset)
    
    for img in unseen_img_list:
        unseen_input_img = cv2.imread(UNSEEN_DATA_PATH + '/'+ dataset + '/'+ img )
        unseen_input_img_resize = cv2.resize(unseen_input_img,(img_rows, img_cols))            
        unseen_img_data_list.append(unseen_input_img_resize) 

unseen_num_classes = len(unseen_img_classes_names_list)
print(unseen_num_classes)



['test']
Loading images from test folder

1


## Approach1: Unseen Image preprocessiong

In [55]:
unseen_img_data = np.array(unseen_img_data_list)
unseen_img_data = unseen_img_data.astype('float16')
unseen_img_data /= 255

print(unseen_img_data.shape)

unseen_num_of_samples = unseen_img_data.shape[0]
print(unseen_num_of_samples)

unseen_input_shape = unseen_img_data[0].shape
print(unseen_input_shape)

X_unseen = unseen_img_data



(1, 300, 300, 3)
1
(300, 300, 3)


In [56]:
# Y_pred_unseen_loaded = loaded_model.predict(X_unseen)
# print(Y_pred_unseen_loaded)

# y_pred_unseen_loaded = np.argmax(Y_pred_unseen_loaded, axis=1)
# print("Class:")
# print(y_pred_unseen_loaded)

In [57]:
loaded_model_augmentation_copy

Y_pred_unseen_loaded_aug = loaded_model_augmentation_copy.predict(X_unseen)
print(Y_pred_unseen_loaded_aug)

y_pred_unseen_loaded_aug = np.argmax(Y_pred_unseen_loaded_aug, axis=1)
print("Class:")
print(y_pred_unseen_loaded_aug)

[[3.4806491e-03 9.6446836e-01 7.3256488e-06 2.2792650e-11 3.4116962e-07
  1.7729743e-08 5.6952922e-07 2.6314658e-15 3.1912953e-02 1.2978201e-04]]
Class:
[1]


**Saving and loading model and weights**

In [58]:
#loaded_model.save('loaded_model_augmentation_copy.hdf5')
#loaded_model_augmentation_copy = keras.models.load_model('loaded_model_augmentation_copy.hdf5')

## Approach2: Without Augmentation

### Shuffling the dataset

In [59]:
X, Y = shuffle(img_data, classes, random_state=2)

### Splitting the dataset into train and test

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [61]:
X_train.shape

(2980, 300, 300, 3)

In [62]:
y_train.shape

(2980, 10)

In [63]:
X_test.shape

(745, 300, 300, 3)

In [64]:
y_test.shape

(745, 10)

In [65]:
del X,Y

### Defining the model

In [66]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D

In [67]:
model_without_augmentation = Sequential()

model_without_augmentation.add(Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
model_without_augmentation.add(Conv2D(32, (2, 2), activation='relu'))
model_without_augmentation.add(MaxPooling2D(pool_size=(2, 2)))
model_without_augmentation.add(Dropout(0.5))

model_without_augmentation.add(Conv2D(64, (2, 2), activation='relu'))
model_without_augmentation.add(Conv2D(64, (2, 2), activation='relu'))
model_without_augmentation.add(MaxPooling2D(pool_size=(2, 2)))
model_without_augmentation.add(Dropout(0.5))

model_without_augmentation.add(Conv2D(128, (2, 2), activation='relu'))
model_without_augmentation.add(Conv2D(128, (2, 2), activation='relu'))
model_without_augmentation.add(MaxPooling2D(pool_size=(2, 2)))
model_without_augmentation.add(Dropout(0.5))

model_without_augmentation.add(Conv2D(256, (2, 2), activation='relu'))
model_without_augmentation.add(Conv2D(256, (2, 2), activation='relu'))
model_without_augmentation.add(MaxPooling2D(pool_size=(2, 2)))
model_without_augmentation.add(Dropout(0.5))

# model_without_augmentation.add(Conv2D(512, (2, 2), activation='relu'))
# model_without_augmentation.add(Conv2D(512, (2, 2), activation='relu'))
# model_without_augmentation.add(MaxPooling2D(pool_size=(2, 2)))
# model_without_augmentation.add(Dropout(0.5))

model_without_augmentation.add(Flatten())
model_without_augmentation.add(Dense(256, activation='relu'))
model_without_augmentation.add(Dropout(0.5))
model_without_augmentation.add(Dense(num_classes, activation='softmax'))

### Compiling the model

In [68]:
model_without_augmentation.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

### Model Summary

In [69]:
model_without_augmentation.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 299, 299, 32)      416       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 298, 298, 32)      4128      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 148, 148, 64)      8256      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 147, 147, 64)      16448     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 64)        0         
__________

In [70]:
model_without_augmentation.get_config()

{'name': 'sequential_2',
 'layers': [{'class_name': 'Conv2D',
   'config': {'name': 'conv2d_9',
    'trainable': True,
    'batch_input_shape': (None, 300, 300, 3),
    'dtype': 'float32',
    'filters': 32,
    'kernel_size': (2, 2),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d_10',
    'trainable': True,
    'filters': 32,
    'kernel_size': (2, 2),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',


In [71]:
model_without_augmentation.layers[0].get_config()

{'name': 'conv2d_9',
 'trainable': True,
 'batch_input_shape': (None, 300, 300, 3),
 'dtype': 'float32',
 'filters': 32,
 'kernel_size': (2, 2),
 'strides': (1, 1),
 'padding': 'valid',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [72]:
model_without_augmentation.layers

In [73]:
model_without_augmentation.weights

[<tf.Variable 'conv2d_9/kernel:0' shape=(2, 2, 3, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_9/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_10/kernel:0' shape=(2, 2, 32, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_10/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_11/kernel:0' shape=(2, 2, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_11/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_12/kernel:0' shape=(2, 2, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_12/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_13/kernel:0' shape=(2, 2, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2d_13/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2d_14/kernel:0' shape=(2, 2, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2d_14/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2d_15/kernel:0' shape=(2, 2, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv2d_15/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Var

In [74]:
model_without_augmentation.layers[0].get_weights()

[array([[[[ 0.05597378,  0.08270033, -0.19130701,  0.18103497,
            0.0217277 ,  0.19240512,  0.16087823,  0.01111196,
           -0.09873318,  0.01175864,  0.16114394,  0.02243528,
           -0.03742471, -0.20510697, -0.05359644,  0.1384847 ,
           -0.16788596,  0.02901481, -0.13130733,  0.16015334,
           -0.04992485, -0.19933666, -0.06518777,  0.00418131,
           -0.08779145, -0.18756044, -0.13579378, -0.03455636,
            0.11150478,  0.12366886, -0.05686542, -0.04363944],
          [-0.14506951, -0.07634227,  0.14257641, -0.07086608,
           -0.18159507, -0.1701147 ,  0.15696116, -0.17163263,
           -0.19672674,  0.15963109,  0.19665484,  0.01777773,
           -0.09146709,  0.11065884,  0.15861364,  0.20298366,
            0.14175205, -0.1646042 ,  0.12849797,  0.03832509,
            0.19647582, -0.18811591,  0.10165577, -0.17093349,
           -0.1545932 ,  0.10074   , -0.08499116,  0.05600591,
            0.14255248, -0.07654078, -0.01550062, -0.0

In [75]:
# For layer 0, checking the filter (get_weights()[0]) and bias (get_weights()[0]))

print(type(model_without_augmentation.layers[0].get_weights()))

print(len(model_without_augmentation.layers[0].get_weights()))

print(type(model_without_augmentation.layers[0].get_weights()[0]))

print(model_without_augmentation.layers[0].get_weights()[0].shape)

print(type(model_without_augmentation.layers[0].get_weights()[1]))

print(model_without_augmentation.layers[0].get_weights()[1].shape)

<class 'list'>
2
<class 'numpy.ndarray'>
(2, 2, 3, 32)
<class 'numpy.ndarray'>
(32,)


In [76]:
#Checking input shape for layer 0

model_without_augmentation.layers[0].input_shape

(None, 300, 300, 3)

In [77]:
#Checking output shape for layer 0

model_without_augmentation.layers[0].output_shape

(None, 299, 299, 32)

In [78]:
model_without_augmentation.layers[0].trainable

True

### Approach2: Training/fitting the model without augmentation

In [79]:
num_epoch = 1
batch_size = 32

# hist = model_without_augmentation.fit(X_train, y_train, batch_size = batch_size, 
#                  epochs = num_epoch, verbose = 1, 
#                  validation_data = (X_test, y_test))
# # Total number of epoch = 15

#### Saving and loading model and weights

In [80]:
#model_without_augmentation.save('model_without_augmentation_V1.hdf5')
#model_without_augmentation_15_epochs = keras.models.load_model('model_without_augmentation_V1.hdf5')

In [81]:
#model_without_augmentation.save('model_without_augmentation_V2.hdf5')
model_without_augmentation_16_epochs = keras.models.load_model('model_without_augmentation_V2.hdf5')

### Approach2: Evaluating the model

In [82]:
# #loaded_model_augmentation_copy
# #num_epoch = 10
# batch_size_wo_aug = 32

# score_without_augmentation = model_without_augmentation_15_epochs.evaluate(X_test, y_test, batch_size=batch_size_wo_aug)

# print('Test Loss:', score_without_augmentation[0])
# print('Test Accuracy:', score_without_augmentation[1])

In [83]:
#loaded_model_augmentation_copy
#num_epoch = 10
batch_size_wo_aug = 32

score_without_augmentation = model_without_augmentation_16_epochs.evaluate(X_test, y_test, batch_size=batch_size_wo_aug)

print('Test Loss:', score_without_augmentation[0])
print('Test Accuracy:', score_without_augmentation[1])

745/745 [==============================] - 5s 7ms/step
Test Loss: 0.5930968964259896
Test Accuracy: 0.8348993289390666


In [84]:
test_image_wo_aug = X_test[0:1]
print (test_image_wo_aug.shape)

(1, 300, 300, 3)


In [85]:
print(model_without_augmentation_16_epochs.predict(test_image_wo_aug))
print(model_without_augmentation_16_epochs.predict_classes(test_image_wo_aug))
print(y_test[0:1])

[[1.44567423e-07 1.12599926e-04 4.76738296e-06 7.30551106e-11
  1.16827963e-04 6.02427761e-08 9.52838036e-06 2.75178929e-04
  2.65845854e-04 9.99215007e-01]]
[9]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Approach2: Predict and compute the confusion matrix

In [86]:
#from sklearn.metrics import confusion_matrix

Y_pred_wo_aug = model_without_augmentation_16_epochs.predict(X_test)
print(Y_pred_wo_aug)

[[1.4456742e-07 1.1259982e-04 4.7673966e-06 ... 2.7517919e-04
  2.6584585e-04 9.9921501e-01]
 [9.8393439e-06 3.2678051e-03 3.0634372e-05 ... 2.3150661e-04
  9.7985327e-01 1.6570099e-02]
 [9.9868566e-01 8.8935536e-05 3.9720530e-07 ... 1.1792540e-03
  8.4127498e-07 1.3961659e-07]
 ...
 [9.6588959e-09 1.9726602e-03 7.0673245e-06 ... 3.0545116e-04
  1.6819090e-06 1.1222236e-06]
 [1.8546119e-04 5.1752027e-06 2.2509077e-09 ... 1.2843721e-05
  5.2417265e-10 1.6792757e-06]
 [3.0685840e-08 7.7340178e-07 5.2590160e-10 ... 6.2453279e-11
  6.1961209e-08 7.6763048e-05]]


In [87]:
y_pred_wo_aug = np.argmax(Y_pred_wo_aug, axis=1)
print(y_pred_wo_aug)

# Class description:
# 0 = airplanes
# 1 = BACKGROUND_Google
# 2 = bonsai
# 3 = car_side
# 4 = Faces
# 5 = Faces_easy
# 6 = grand_piano
# 7 = Leopards
# 8 = Motorbikes
# 9 = watch

[9 8 0 0 9 2 0 5 8 8 7 7 0 0 4 7 8 0 8 5 4 4 7 7 5 7 1 0 1 5 5 7 5 7 4 7 5
 4 1 3 1 8 8 7 6 4 0 4 4 1 0 5 7 3 7 8 9 0 0 8 1 5 0 4 8 4 5 5 5 1 9 8 7 4
 2 1 8 3 1 8 0 0 4 0 8 8 5 0 8 0 1 4 5 4 0 5 1 4 8 6 8 0 1 0 8 7 5 4 8 5 5
 0 5 8 8 4 8 5 8 0 9 8 7 9 2 0 5 5 4 4 7 4 3 8 3 8 8 8 8 8 0 4 0 8 0 0 7 4
 8 0 7 0 5 0 5 5 5 0 5 1 5 3 8 4 5 8 4 1 0 4 0 0 8 1 4 5 5 8 3 8 1 5 1 7 0
 0 8 8 7 9 0 3 8 0 0 3 0 5 8 4 0 7 4 8 3 8 8 0 8 4 8 4 6 3 1 7 8 7 5 2 4 7
 7 0 8 7 8 4 5 5 0 4 0 0 5 7 4 0 8 0 4 0 7 8 9 2 8 2 0 1 1 7 9 1 0 1 0 0 8
 8 0 2 3 0 0 0 9 0 8 4 7 7 8 4 8 4 5 7 0 8 6 5 3 4 8 5 5 0 0 8 8 5 7 8 8 6
 5 8 0 0 7 0 7 8 0 7 1 0 8 7 8 0 0 7 1 9 7 8 0 5 8 2 1 0 1 1 0 7 0 3 0 0 7
 3 4 9 8 8 7 0 5 0 4 4 0 4 4 6 1 9 4 0 8 0 5 4 8 0 0 0 8 8 0 8 3 8 5 7 0 1
 1 8 5 7 7 8 8 1 8 1 5 8 1 8 8 0 7 7 1 0 0 8 8 8 9 3 0 9 8 9 6 0 8 0 8 3 4
 9 5 7 0 0 7 0 7 8 8 8 0 5 8 4 2 1 9 0 7 8 8 3 5 4 8 0 0 7 1 9 8 1 5 5 7 7
 5 0 7 0 1 0 8 8 0 7 0 1 4 8 0 5 7 0 7 0 1 5 4 8 0 1 0 6 1 2 8 2 2 7 0 7 5
 5 0 8 9 4 5 8 3 8 0 0 1 

In [88]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_wo_aug))

[[149   2   0   0   0   0   0  18   2   0]
 [  3  53  11   3   1   0   2  10   4   3]
 [  1   4   7   1   0   0   1   7   0   0]
 [  0   0   0  19   0   0   0   3   1   0]
 [  2   7   0   0  73   3   0   7   0   0]
 [  0   0   1   0   0  94   0   0   0   1]
 [  1   0   3   0   0   0   6   2   0   0]
 [  0   0   0   0   0   0   0  40   0   0]
 [  0   1   2   0   0   0   0   0 149   0]
 [  3   3   1   0   0   0   0   5   4  32]]


## Approach2: Prediction on unseen image

In [89]:


# UNSEEN_DATA_PATH = os.path.join(PATH, 'useen_image')

# #print(UNSEEN_DATA_PATH)

# unseen_data_dir_list = os.listdir(UNSEEN_DATA_PATH)

# if '.DS_Store' in data_dir_list :
#     data_dir_list.remove('.DS_Store')
    
# print(unseen_data_dir_list)

# unseen_img_data_list=[]
# unseen_img_classes_names_list=[]

# for dataset in unseen_data_dir_list:
    
#     unseen_img_classes_names_list.append(dataset) 
    
#     print ('Loading images from {} folder\n'.format(dataset)) 
    
#     unseen_img_list = os.listdir(UNSEEN_DATA_PATH+'/'+ dataset)
    
#     for img in unseen_img_list:
#         unseen_input_img = cv2.imread(UNSEEN_DATA_PATH + '/'+ dataset + '/'+ img )
#         unseen_input_img_resize = cv2.resize(unseen_input_img,(img_rows, img_cols))            
#         unseen_img_data_list.append(unseen_input_img_resize) 

# unseen_num_classes = len(unseen_img_classes_names_list)
# print(unseen_num_classes)



## Approach2: Unseen Image preprocessiong

In [90]:
# unseen_img_data = np.array(unseen_img_data_list)
# unseen_img_data = unseen_img_data.astype('float16')
# unseen_img_data /= 255

# print(unseen_img_data.shape)

# unseen_num_of_samples = unseen_img_data.shape[0]
# print(unseen_num_of_samples)

# unseen_input_shape = unseen_img_data[0].shape
# print(unseen_input_shape)

# X_unseen = unseen_img_data



In [91]:
Y_pred_unseen_wo_aug = model_without_augmentation_16_epochs.predict(X_unseen)
print(Y_pred_unseen_wo_aug)

y_pred_unseen_wo_aug = np.argmax(Y_pred_unseen_wo_aug, axis=1)
print("Class:")
print(y_pred_unseen_wo_aug)

[[5.62765035e-06 2.43111640e-01 1.67283579e-04 3.19196224e-05
  3.68486726e-05 1.01399342e-08 6.76532928e-03 1.49181066e-03
  1.03218846e-01 6.45170689e-01]]
Class:
[9]


## Approach3: Transfer learning using VGG Architecture

In [92]:
from keras.layers import Input, Dense

In [93]:
# Custom_vgg_model_1

#Training the classifier alone
num_channel = 3
img_rows_vgg = 224
img_cols_vgg = 224

image_input = Input(shape=(img_rows_vgg, img_cols_vgg, num_channel))

In [94]:
from keras.applications.vgg16 import VGG16

model_VGG = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

In [95]:
model_VGG.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [96]:
last_layer_VGG = model_VGG.get_layer('fc2').output
out_VGG = Dense(num_classes, activation='softmax', name='output')(last_layer_VGG)

In [97]:
from keras.models import Model

custom_vgg_model = Model(image_input, out_VGG)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [98]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [99]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [100]:
img_rows=224
img_cols=224

img_data_list=[]
classes_names_list=[]

for dataset in data_dir_list:
    
    classes_names_list.append(dataset) 
    
    print ('Loading images from {} folder\n'.format(dataset)) 
    
    img_list = os.listdir(DATA_PATH+'/'+ dataset)
    
    for img in img_list:
        input_img = cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))            
        img_data_list.append(input_img_resize)

Loading images from airplanes folder

Loading images from BACKGROUND_Google folder

Loading images from bonsai folder

Loading images from car_side folder

Loading images from Faces folder

Loading images from Faces_easy folder

Loading images from grand_piano folder

Loading images from Leopards folder

Loading images from Motorbikes folder

Loading images from watch folder



### Approach3: Getting the number of classes

In [101]:
num_classes = len(classes_names_list)
print(num_classes)

10


### Approach3: Image preprocessiong

In [102]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float16')
img_data /= 255              ############### 0-255, i.e., 2**8 = 256 possibilities is the rance for each individual colour. 
                             ############### By dividing by 255, we get values in the 0.0-1.0 range (normalization) 
                             ############### NN or deep learning required scaled or normalized data


In [103]:
print (img_data.shape)

(3725, 224, 224, 3)


In [104]:
num_of_samples = img_data.shape[0]
print(num_of_samples)

3725


In [105]:
input_shape = img_data[0].shape
print(input_shape)

(224, 224, 3)


In [106]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:800]=0
classes[800:1268]=1
classes[1268:1396]=2
classes[1396:1519]=3
classes[1519:1954]=4
classes[1954:2389]=5
classes[2389:2488]=6
classes[2488:2688]=7
classes[2688:3486]=8
classes[3468:]=9

print(classes)

[0 0 0 ... 9 9 9]


### Approach3: Converting class labels to numberic using on-hot encoding

In [107]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

print(classes)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


### Approach3: Shuffling the dataset

In [108]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

### Approach3: Splitting the dataset into train and test

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [110]:
X_train.shape

(2980, 224, 224, 3)

In [111]:
y_train.shape

(2980, 10)

In [112]:
X_test.shape

(745, 224, 224, 3)

In [113]:
y_test.shape

(745, 10)

In [114]:
del X,Y

In [115]:
# custom_vgg_model.fit(X_train, y_train, batch_size=batch_size, epochs=2, verbose=1, 
#                      validation_data=(X_test, y_test))

In [116]:
#custom_vgg_model.save('custom_vgg_model_V1.hdf5')
custom_vgg_model_6_epochs = keras.models.load_model('custom_vgg_model_V1.hdf5')

In [117]:
# custom_vgg_model_6_epochs.fit(X_train, y_train, batch_size=batch_size, epochs=2, verbose=1, 
#                      validation_data=(X_test, y_test))

In [118]:
#custom_vgg_model_6_epochs.save('custom_vgg_model_V2.hdf5')
custom_vgg_model_12_epochs = keras.models.load_model('custom_vgg_model_V2.hdf5')

## Approach3: Prediction on unseen image

In [119]:
UNSEEN_DATA_PATH = os.path.join(PATH, 'useen_image')

print(UNSEEN_DATA_PATH)

unseen_data_dir_list = os.listdir(UNSEEN_DATA_PATH)

if '.DS_Store' in data_dir_list :
    data_dir_list.remove('.DS_Store')
    
print(unseen_data_dir_list)
img_rows_vgg = 224
img_cols_vgg = 224
unseen_img_data_list_vgg=[]
unseen_img_classes_names_list_vgg=[]

for dataset in unseen_data_dir_list:
    
    unseen_img_classes_names_list_vgg.append(dataset) 
    
    print ('Loading images from {} folder\n'.format(dataset)) 
    
    unseen_img_list_vgg = os.listdir(UNSEEN_DATA_PATH+'/'+ dataset)
    
    for img in unseen_img_list_vgg:
        unseen_input_img_vgg = cv2.imread(UNSEEN_DATA_PATH + '/'+ dataset + '/'+ img )
        unseen_input_img_resize_vgg = cv2.resize(unseen_input_img_vgg,(img_rows_vgg, img_cols_vgg))            
        unseen_img_data_list_vgg.append(unseen_input_img_resize_vgg) 


C:\Work\INSOFE\Study Material\CUTe\CUTe4\Image classification\useen_image
['test']
Loading images from test folder



In [120]:
unseen_num_classes_vgg = len(unseen_img_classes_names_list_vgg)
print(unseen_num_classes_vgg)

1


## Approach3: Unseen Image preprocessiong

In [121]:
unseen_img_data_vgg = np.array(unseen_img_data_list_vgg)
unseen_img_data_vgg = unseen_img_data_vgg.astype('float16')
unseen_img_data_vgg /= 255

In [122]:
print(unseen_img_data_vgg.shape)

(1, 224, 224, 3)


In [123]:
unseen_num_of_samples_vgg = unseen_img_data_vgg.shape[0]
print(unseen_num_of_samples_vgg)

1


In [124]:
unseen_input_shape_vgg = unseen_img_data_vgg[0].shape
print(unseen_input_shape_vgg)

(224, 224, 3)


In [125]:
X_unseen = unseen_img_data_vgg

In [126]:
custom_vgg_model_6_epochs = keras.models.load_model('custom_vgg_model_V1.hdf5')

In [127]:
Y_pred_unseen_vgg = custom_vgg_model_6_epochs.predict(X_unseen)
print(Y_pred_unseen_vgg)

[[8.4538562e-03 8.1938523e-01 1.2409748e-02 9.5577724e-09 1.0089910e-02
  2.4711937e-03 5.4239383e-04 7.3763334e-07 1.2336677e-01 2.3280153e-02]]


In [128]:
Y_pred_unseen_vgg_2 = custom_vgg_model_12_epochs.predict(X_unseen)
print(Y_pred_unseen_vgg_2)

[[1.5328033e-05 9.9052715e-01 4.7544770e-05 5.4770674e-13 2.2283277e-07
  1.3863541e-03 2.0910466e-06 1.2011618e-09 2.4271107e-03 5.5941734e-03]]


In [129]:
y_pred_unseen_vgg = np.argmax(Y_pred_unseen_vgg, axis=1)
print('Class: ')
print(y_pred_unseen_vgg)

Class: 
[1]


In [130]:
y_pred_unseen_vgg_2 = np.argmax(Y_pred_unseen_vgg_2, axis=1)
print('Class: ')
print(y_pred_unseen_vgg_2)

Class: 
[1]


### Approach 4 Transfer learning using ResNet50 Architecture 

In [131]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Input, Dense

In [132]:
num_channel = 3
img_rows=300
img_cols=300

image_input = Input(shape=(img_rows, img_cols, num_channel))

resnet = ResNet50(input_tensor=image_input, include_top=True, weights='imagenet')

In [133]:
resnet = ResNet50(input_tensor=image_input, include_top=True, weights='imagenet')

In [134]:
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [135]:
last_layer = resnet.get_layer('fc1000').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [136]:
from keras.models import Model

custom_resnet_model = Model(image_input, out)
custom_resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [137]:
for layer in custom_resnet_model.layers[:-1]:
    layer.trainable = False

In [138]:
custom_resnet_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [139]:
img_rows=300
img_cols=300

img_data_list=[]
classes_names_list=[]

for dataset in data_dir_list:
    
    classes_names_list.append(dataset) 
    
    print ('Loading images from {} folder\n'.format(dataset)) 
    
    img_list = os.listdir(DATA_PATH+'/'+ dataset)
    
    for img in img_list:
        input_img = cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))            
        img_data_list.append(input_img_resize)

Loading images from airplanes folder

Loading images from BACKGROUND_Google folder

Loading images from bonsai folder

Loading images from car_side folder

Loading images from Faces folder

Loading images from Faces_easy folder

Loading images from grand_piano folder

Loading images from Leopards folder

Loading images from Motorbikes folder

Loading images from watch folder



### Approach4: Getting the number of classes

In [140]:
num_classes = len(classes_names_list)
print(num_classes)

10


### Approach4: Image preprocessiong

In [141]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float16')
img_data /= 255              ############### 0-255, i.e., 2**8 = 256 possibilities is the rance for each individual colour. 
                             ############### By dividing by 255, we get values in the 0.0-1.0 range (normalization) 
                             ############### NN or deep learning required scaled or normalized data


In [142]:
print (img_data.shape)

(3725, 300, 300, 3)


In [143]:
num_of_samples = img_data.shape[0]
print(num_of_samples)

3725


In [144]:
input_shape = img_data[0].shape
print(input_shape)

(300, 300, 3)


In [145]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:800]=0
classes[800:1268]=1
classes[1268:1396]=2
classes[1396:1519]=3
classes[1519:1954]=4
classes[1954:2389]=5
classes[2389:2488]=6
classes[2488:2688]=7
classes[2688:3486]=8
classes[3468:]=9

print(classes)

[0 0 0 ... 9 9 9]


### Approach4: Converting class labels to numberic using on-hot encoding

In [146]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

print(classes)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


### Approach4: Shuffling the dataset

In [147]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

### Approach4: Splitting the dataset into train and test

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [149]:
X_train.shape

(2980, 300, 300, 3)

In [150]:
y_train.shape

(2980, 10)

In [151]:
X_test.shape

(745, 300, 300, 3)

In [152]:
y_test.shape

(745, 10)

In [153]:
del X,Y

In [154]:
# custom_resnet_model.fit(X_train, y_train, batch_size=batch_size, epochs=5, verbose=1, 
#                      validation_data=(X_test, y_test))

In [155]:
#custom_resnet_model.save('custom_resnet_model_V1.hdf5')
custom_resnet_model_5_epochs = keras.models.load_model('custom_resnet_model_V1.hdf5')

In [156]:
# custom_resnet_model_5_epochs.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=1, 
#                      validation_data=(X_test, y_test))

In [157]:
#custom_resnet_model_5_epochs.save('custom_resnet_model_V2.hdf5')
custom_resnet_model_20_epochs = keras.models.load_model('custom_resnet_model_V2.hdf5')

## Approach4: Prediction on unseen data

In [158]:
# UNSEEN_DATA_PATH = os.path.join(PATH, 'useen_image')

# print(UNSEEN_DATA_PATH)

In [159]:
# unseen_data_dir_list = os.listdir(UNSEEN_DATA_PATH)

# if '.DS_Store' in data_dir_list :
#     data_dir_list.remove('.DS_Store')
    
# print(unseen_data_dir_list)

In [160]:
# img_rows = 300
# img_cols = 300

# unseen_img_data_list=[]
# unseen_img_classes_names_list=[]

# for dataset in unseen_data_dir_list:
    
#     unseen_img_classes_names_list.append(dataset) 
    
#     print ('Loading images from {} folder\n'.format(dataset)) 
    
#     unseen_img_list = os.listdir(UNSEEN_DATA_PATH+'/'+ dataset)
    
#     for img in unseen_img_list:
#         unseen_input_img = cv2.imread(UNSEEN_DATA_PATH + '/'+ dataset + '/'+ img )
#         unseen_input_img_resize = cv2.resize(unseen_input_img,(img_rows, img_cols))            
#         unseen_img_data_list.append(unseen_input_img_resize) 

In [161]:
# unseen_num_classes = len(unseen_img_classes_names_list)
# print(unseen_num_classes)

## Unseen Image preprocessiong

In [162]:
# unseen_img_data = np.array(unseen_img_data_list)
# unseen_img_data = unseen_img_data.astype('float16')
# unseen_img_data /= 255

In [163]:
# print(unseen_img_data.shape)

In [164]:
# unseen_num_of_samples = unseen_img_data.shape[0]
# print(unseen_num_of_samples)

In [165]:
# unseen_input_shape = unseen_img_data[0].shape
# print(unseen_input_shape)

In [166]:
X_unseen = unseen_img_data

In [167]:
Y_pred_unseen_resnet = custom_resnet_model_20_epochs.predict(X_unseen)
print(Y_pred_unseen_resnet)

[[0.16106714 0.15897281 0.02882703 0.02258698 0.10685999 0.0911625
  0.02275665 0.03600837 0.301825   0.06993349]]


In [168]:
y_pred_unseen_resnet = np.argmax(Y_pred_unseen_resnet, axis=1)
print("Class: ")
print(y_pred_unseen_resnet)

Class: 
[8]


## Class description:
    0 = airplanes
    1 = BACKGROUND_Google
    2 = bonsai
    3 = car_side
    4 = Faces
    5 = Faces_easy
    6 = grand_piano
    7 = Leopards
    8 = Motorbikes
    9 = watch

# Results

In [302]:
# print("Class predicted by model with augmented data: ", y_pred_unseen_loaded_aug)
# print("Class predicted by model without augmented data: ", y_pred_unseen_wo_aug)
# print("Class predicted by transfer learning VGG16 model(6 epochs): ", y_pred_unseen_vgg)
# print("Class predicted by transfer learning VGG16 model(12 epochs): ", y_pred_unseen_vgg_2)
# print("Class predicted by transfer learning ResNet50 model: ", y_pred_unseen_resnet)

In [303]:
# print("Class wise probabilities for the unseen image predicted by model with augmented data: \n", Y_pred_unseen_loaded_aug)
# print()
# print("Class wise probabilities for the unseen image predicted by model without augmented data: \n", Y_pred_unseen_wo_aug)
# print()
# print("Class wise probabilities for the unseen image predicted by transfer learning VGG16 model (6 epochs): \n", Y_pred_unseen_vgg)
# print()
# print("Class wise probabilities for the unseen image predicted by transfer learning VGG16 model (12 epochs): \n", Y_pred_unseen_vgg_2)
# print()
# print("Class wise probabilities for the unseen image predicted by transfer learning ResNet50 model: \n", Y_pred_unseen_resnet)

In [301]:
# Class_location ={"[0]" : "airplanes", "[1]" : "BACKGROUND_Google", "[2]" : "bonsai", "[3]" : "car_side", "[4]" : "Faces", \
#                  "[5]" : "Faces_easy", "[6]" : "grand_piano", "[7]" : "Leopards", "[8]" : "Motorbikes", "[9]" : "watch"}


# print("Model with augmented data: \nPredicted Class: ", Class_location.get(str(y_pred_unseen_loaded_aug)), "\nWith probability: ",np.amax(Y_pred_unseen_loaded_aug))
# print()
# print("Model without augmented data: \nPredicted Class: ", Class_location.get(str(y_pred_unseen_wo_aug)), "\nWith probability: ",np.amax(Y_pred_unseen_wo_aug))
# print()
# print("Transfer learning VGG16 model (6 epochs): \nPredicted Class: ", Class_location.get(str(y_pred_unseen_vgg)), "\nWith probability: ",np.amax(Y_pred_unseen_vgg))
# print()
# print("Transfer learning VGG16 model (12 epochs): \nPredicted Class: ", Class_location.get(str(y_pred_unseen_vgg_2)), "\nWith probability: ",np.amax(Y_pred_unseen_vgg_2))
# print()
# print("Transfer learning ResNet50 model: \nPredicted Class: ", Class_location.get(str(y_pred_unseen_resnet)), "\nWith probability: ",np.amax(Y_pred_unseen_resnet))

In [213]:
import pandas as pd

In [267]:
data = {'Class_Code': ['0','1','2','3','4','5','6','7','8','9'], \
        'Class_name': ['airplanes', 'BACKGROUND_Google', 'bonsai', 'car_side', 'Faces', 'Faces_easy', 'grand_piano', \
                       'Leopards', 'Motorbikes', 'watch'], \
       'Model with augmented data': [str(Y_pred_unseen_loaded_aug[0][i]) for i in range(len(Y_pred_unseen_loaded_aug[0]))],\
        'Model without augmented data': [str(Y_pred_unseen_wo_aug[0][i]) for i in range(len(Y_pred_unseen_wo_aug[0]))],\
        'Transfer learning VGG16 model (6 epochs)': [str(Y_pred_unseen_vgg[0][i]) for i in range(len(Y_pred_unseen_vgg[0]))],\
        'Transfer learning VGG16 model (12 epochs)': [str(Y_pred_unseen_vgg_2[0][i]) for i in range(len(Y_pred_unseen_vgg_2[0]))],\
        'Transfer learning ResNet50 model': [str(Y_pred_unseen_resnet[0][i]) for i in range(len(Y_pred_unseen_resnet[0]))]}
Final_Output =pd.DataFrame.from_dict(data)#, orient='index')
Final_Output

,Class_Code,Class_name,Model with augmented data,Model without augmented data,Transfer learning VGG16 model (6 epochs),Transfer learning VGG16 model (12 epochs),Transfer learning ResNet50 model
0,0,airplanes,0.003480649,5.6276504e-06,0.008453856,1.5328033e-05,0.16106714
1,1,BACKGROUND_Google,0.96446836,0.24311164,0.81938523,0.99052715,0.15897281
2,2,bonsai,7.3256488e-06,0.00016728358,0.012409748,4.754477e-05,0.028827034
3,3,car_side,2.279265e-11,3.1919622e-05,9.557772e-09,5.4770674e-13,0.022586975
4,4,Faces,3.4116962e-07,3.6848673e-05,0.01008991,2.2283277e-07,0.10685999
5,5,Faces_easy,1.7729743e-08,1.0139934e-08,0.0024711937,0.0013863541,0.0911625
6,6,grand_piano,5.695292e-07,0.0067653293,0.0005423938,2.0910466e-06,0.022756653
7,7,Leopards,2.6314658e-15,0.0014918107,7.3763334e-07,1.2011618e-09,0.036008373
8,8,Motorbikes,0.031912953,0.103218846,0.12336677,0.0024271107,0.301825
9,9,watch,0.00012978201,0.6451707,0.023280153,0.0055941734,0.06993349


In [268]:
Final_Output_melt = Final_Output.melt(id_vars=["Class_Code", "Class_name"])

In [299]:
#Final_Output_melt

In [297]:
Final_Output_melt['value'] = Final_Output_melt['value'].apply(lambda x: float(x))
#Final_Output_melt.dtypes

In [300]:
#Final_Output_melt_sorted = Final_Output_melt.sort_values(by='value', ascending=False)
Final_Output_melt_sorted.head()

,Class_Code,Class_name,variable,value
31,1,BACKGROUND_Google,Transfer learning VGG16 model (12 epochs),0.990527
1,1,BACKGROUND_Google,Model with augmented data,0.964468
21,1,BACKGROUND_Google,Transfer learning VGG16 model (6 epochs),0.819385
19,9,watch,Model without augmented data,0.645171
48,8,Motorbikes,Transfer learning ResNet50 model,0.301825


In [298]:
best_model_result = Final_Output_melt[Final_Output_melt.variable == 'Transfer learning VGG16 model (12 epochs)']
best_model_result

,Class_Code,Class_name,variable,value
31,1,BACKGROUND_Google,Transfer learning VGG16 model (12 epochs),9.905272e-01
39,9,watch,Transfer learning VGG16 model (12 epochs),5.594173e-03
38,8,Motorbikes,Transfer learning VGG16 model (12 epochs),2.427111e-03
35,5,Faces_easy,Transfer learning VGG16 model (12 epochs),1.386354e-03
32,2,bonsai,Transfer learning VGG16 model (12 epochs),4.754477e-05
30,0,airplanes,Transfer learning VGG16 model (12 epochs),1.532803e-05
36,6,grand_piano,Transfer learning VGG16 model (12 epochs),2.091047e-06
34,4,Faces,Transfer learning VGG16 model (12 epochs),2.228328e-07
37,7,Leopards,Transfer learning VGG16 model (12 epochs),1.201162e-09
33,3,car_side,Transfer learning VGG16 model (12 epochs),5.477067e-13
